In [1]:
source("~/software/notebook_assist/functions.R")
source("~/software/notebook_assist/packages.R")

setwd("../")
list.files()

[1] "figures"             "input"               "notebooks"          
[4] "source_data"         "supplemental_tables"

In [2]:
setwd("/projects/CARDIPS/analysis/epigenome_resource/analyses/tim/qtl_finemapping/")

In [4]:
caqtl.fs = list.files("CVPC/caqtls",pattern="all_vars", full.names = T)

In [5]:
tmp = fread(caqtl.fs[1],sep="\t",data.table=F)

In [104]:
#!/home/tarthur/software/R-4.0.1/bin/R

source("~/software/notebook_assist/functions.R")
source("~/software/notebook_assist/packages.R")
suppressPackageStartupMessages(library(coloc))
setwd("/projects/CARDIPS/analysis/epigenome_resource/")

args   = commandArgs(trailingOnly=TRUE)
task = as.numeric(args[1])

finemap.fs = fread("analyses/tim/qtl_finemapping/scripts/mapping_file.txt",sep="\t",data.table=F)[task,]

tissue_n_df = fread("analyses/tim/qtl_finemapping/scripts/tissue_n.txt",sep="\t",data.table=F)

qtl_path  = finemap.fs$V1
condition = finemap.fs$V2

data_type = str_extract(qtl_path,  "[a-z]+qtls"  )

tissue    = gsub("/.*","",
                 gsub("/projects/CARDIPS/analysis/epigenome_resource//","",
                 gsub(paste(data_type),"",qtl_path)))

n = tissue_n_df$n[ tissue_n_df$tissue == tissue & tissue_n_df$data_type == data_type]

qtl = fread(qtl_path, sep="\t",data.table=F)
qtl = qtl[ qtl$type == condition, ]
qtl = qtl[ qtl$pos %in% names(table(qtl$pos))[ table(qtl$pos) == 1],]

finemap_df = list(beta = qtl$beta,
         varbeta =  qtl$se**2,
         snp = qtl$id,
         position = qtl$pos,
         type = "quant",
         N = n, 
         MAF = qtl$af)

finemapped = finemap.abf(dataset=finemap_df,  p1 = 1/(nrow(qtl)+1))

finemapped$`Element ID` = unique(qtl$element_id)
finemapped$Condition    = unique(qtl$type)

finemapped = finemapped[ complete.cases(finemapped), ]
finemapped_out = finemapped %>% arrange(-SNP.PP) %>% mutate(`Cumulative PP` = cumsum(SNP.PP)) %>% 
                                mutate(`Credible Set` = ifelse(`Cumulative PP` <= 0.99, "TRUE","FALSE")) %>% 
                                select( snp,position,`Element ID`, Condition, prior, SNP.PP, V.,z.,r.,lABF.,`Credible Set`) %>%
                                rename( Position = position, `SNP ID` = snp, Prior = prior)


finemap_dir=paste0("analyses/tim/qtl_finemapping/",tissue,"/",data_type,"/all_vars/")
if (!file.exists(finemap_dir)) {dir.create(finemap_dir, recursive = TRUE)}
fwrite(finemapped_out,paste0(finemap_dir,unique(qtl$element_id),
                             "_",unique(qtl$type),".txt"),
      sep="\t",row.names=F,quote=F)


### Write Bed files for top SNPs 
finemapped_bed = finemapped_out[,c("SNP ID","Position","Element ID","Condition","SNP.PP")]
finemapped_bed$Chromosome   = paste0("chr",(gsub("VAR_","",str_extract(finemapped_bed$`SNP ID`, "VAR_[0-9]+"))))

finemapped_bed$start = finemapped_bed$Position - 1 
finemapped_bed$qtl_id = paste(finemapped_bed$`Element ID`, finemapped_bed$Condition,sep="_")
finemapped_bed2 = finemapped_bed[ finemapped_bed$SNP.PP >= 0.01,
                                 c("Chromosome","start","Position","SNP.PP","qtl_id")]

bed_dir=paste0("analyses/tim/qtl_finemapping/",tissue,"/",data_type,"/top_beds/")
if (!file.exists(bed_dir)) {dir.create(bed_dir, recursive = TRUE)}

fwrite(finemapped_bed2[ order(finemapped_bed2$Chromosome,finemapped_bed2$start),],
       paste0(bed_dir,unique(qtl$element_id),
                             "_",unique(qtl$type),".bed"),
      sep="\t",row.names=F,quote=F, col.names=F)

Warning message in sdY.est(d$varbeta, d$MAF, d$N):
“estimating sdY from maf and varbeta, please directly supply sdY if known”
